### Imports

In [3]:
import os
import pickle
import numpy as np
from pathlib import Path
import pandas as pd
import geopandas as gpd
import scipy
from dotenv import load_dotenv
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import xgboost as xgb
import pymc as pm
import pytensor
import pytensor.tensor as pt
import pytensor.sparse as sparse
import patsy
import arviz as az
import boto3

DATA_DIR = Path("data")

INDEX_COL = "huc12"

bucket_name = "duke-research"
bucket_prefix = ""
file_name_inputs = "inputs.npz"
file_name_gpkg = "final.gpkg"

load_dotenv()

True

### Load data

#### Retrieve from S3

In [ ]:
OVERWRITE = True

for file_name in [file_name_inputs, file_name_gpkg]:
    if os.path.exists(DATA_DIR / file_name) and not OVERWRITE:
        print(f"✓ {file_name} already exists")
        continue

    print(f"Downloading {file_name} from S3...")
    s3 = boto3.client('s3')
    s3.download_file(bucket_name, file_name, DATA_DIR / file_name)
    print(f"✓ Successfully downloaded {file_name}")


✓ inputs.npz already exists
✓ final.gpkg already exists


In [5]:
final_gdf = gpd.read_file(DATA_DIR /  file_name_gpkg)
loaded_data = np.load(DATA_DIR / file_name_inputs)
W = scipy.sparse.csr_matrix((loaded_data['W_data'], loaded_data['W_indices'], loaded_data['W_indptr']))
X = loaded_data['X']
Z = loaded_data['Z']
y = loaded_data['y']
coords = loaded_data['coords']
pretty_predictor_cols = loaded_data['pretty_predictor_cols']
print(f"Loaded data from disk with {X.shape[0]} rows and {X.shape[1]} columns")

Loaded data from disk with 64220 rows and 154 columns


## Baseline modeling

### Linear model

In [4]:
def baseline_lr(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=827)

    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)

    y_pred_train = lr_model.predict(X_train)
    y_pred_test = lr_model.predict(X_test)

    # Print smallest / largest coefficients as well as 90% range
    coefs = lr_model.coef_
    print("Smallest coefficients:")
    print(coefs[np.argsort(coefs)[:3]])
    print("Largest coefficients:")
    print(coefs[np.argsort(coefs)[-3:]])
    print("90% range of coefficients:")
    print(np.percentile(coefs, 95) - np.percentile(coefs, 5))

    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)

    print(f"Linear Regression R² on training set: {r2_train:.4f}")
    print(f"Linear Regression R² on test set: {r2_test:.4f}")

print("Linear baseline model (predictors X):")
baseline_lr(X, y)

print("\nLinear baseline model (embeddings Z):")
baseline_lr(Z, y)


Linear baseline model (predictors X):
Smallest coefficients:
[-8.94796198 -3.34667055 -2.0773022 ]
Largest coefficients:
[2.69161593 4.70096494 5.28841285]
90% range of coefficients:
2.5990572982445936
Linear Regression R² on training set: 0.3272
Linear Regression R² on test set: 0.3266

Linear baseline model (embeddings Z):
Smallest coefficients:
[-1.40754657 -0.47516076 -0.4404622 ]
Largest coefficients:
[0.36082777 0.46202153 0.66120501]
90% range of coefficients:
1.2200746171583043
Linear Regression R² on training set: 0.0893
Linear Regression R² on test set: 0.0925


In [ ]:
def baseline_xgb(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=827)

        # Convert to DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train, )
    dtest = xgb.DMatrix(X_test, label=y_test)

    params = {
        'objective': 'reg:squarederror',
        'max_depth': 6,
        'eta': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'seed': 827  
    }

    num_boost_round = 100
    xgb_model = xgb.train(params, dtrain, num_boost_round)

    y_pred_xgb_train = xgb_model.predict(dtrain)
    y_pred_xgb_test = xgb_model.predict(dtest)

    r2_xgb_train = r2_score(y_train, y_pred_xgb_train)
    r2_xgb_test = r2_score(y_test, y_pred_xgb_test)

    print(f"XGBoost R² on training set: {r2_xgb_train:.4f}")
    print(f"XGBoost R² on test set: {r2_xgb_test:.4f}")

print("XGBoost baseline model (predictors X):")
baseline_xgb(X, y)
print("\nXGBoost baseline model (embeddings Z):")
baseline_xgb(Z, y)

XGBoost baseline model (predictors X):


### Declare model

In [6]:
USE_GAM          = False
RANDOM_SUBSAMPLE = True
USE_GP           = True
TRACK_MU         = True
MULTILEVEL_BETA  = True
ADD_INTERCEPT    = False
USE_EMBED        = True
FLOAT_FORMAT     = "float32"
HUC_LEVELS       = [2, 4] 

# HSGP settings; only used if USE_GP is True
m = 60
c = 1.5

n = len(y)
p = X.shape[1]

pytensor.config.floatX = FLOAT_FORMAT

y = y.astype(FLOAT_FORMAT) # Response variable, shape (n,)
X = X.astype(FLOAT_FORMAT) # Covariates, shape (n, p)
W = W.astype(FLOAT_FORMAT) # Adj matrix (unused, shape (n,n))
Z = Z.astype(FLOAT_FORMAT) # Embeddings, shape (n, embed_dim)
coords = coords.astype(FLOAT_FORMAT) # locations of rows for spatial random effect (n,2) 

if ADD_INTERCEPT:

    # Check to make sure no constant column is present
    if np.all(np.abs(X.mean(axis=0)) < 1e-6):
        X = np.concatenate([np.ones((X.shape[0], 1), dtype=FLOAT_FORMAT), X], axis=1)
        p = X.shape[1]
        pretty_predictor_cols = np.concatenate([["intercept"], pretty_predictor_cols], axis=0)

print(f"Preparing to run model inference with {X.shape[0]} samples and {X.shape[1]} predictors")
print(f"Range of response values is {y.min()} to {y.max()}")
print(f"Range of predictor values is {X.min()} to {X.max()}")

if RANDOM_SUBSAMPLE:
    is_used = np.random.rand(X.shape[0]) < 0.1
    n = is_used.sum()
else:
    is_used = np.ones(X.shape[0], dtype=bool)
    n = len(y)

final_gdf_subset = final_gdf.loc[is_used].copy()
bigger_hucs = sum([final_gdf_subset[f'huc{i}'].unique().tolist() for i in HUC_LEVELS], [])

dims = {
    'predictor': pretty_predictor_cols,
    "obs_id": final_gdf.loc[is_used, INDEX_COL].values,
    "beta_hucs":  ['global_mean'] + bigger_hucs,

}

embed_len = Z.shape[1]

if USE_GAM:
    spline_df = 3
    X_splines = np.stack([patsy.bs(x, df=spline_df) for x in X.T], axis=-1)
    dims['spline_degree']=np.arange(spline_df)


# If the multilevel beta option is used, the coefficients
# are given a multilevel prior in which the huc12 coefficients are grouped
# around a huc8 mean, which is in turn grouped around a huc4 mean
# and those are grouped around a huc2 mean. A global mean is also
# present at the top level. Each covariate gets a different scale parameter
# at each level of the model, the with the scale prior parameter getting
# progressively smaller from top to bottom. The integer-coded huc12, huc8, etc
# are stored as huc12_int and so on in final_gdf.
with pm.Model(coords=dims) as hierarchical_model:

    X_data = pm.Data('X_data', X[is_used], dims=['obs_id', 'predictor'])
    Z_data = pm.Data('Z_data', Z[is_used])

    if MULTILEVEL_BETA:

        # Number of unique HUCs per resolution, starting with the 
        # global mean which is the same across all HUCs.
        unique_per_level = [1] + [len(final_gdf_subset[f'huc{i}'].unique()) for i in HUC_LEVELS]

        n_unique_combined = sum(unique_per_level)
        n_levels = len(unique_per_level)

        cumulative_unique = np.cumsum(unique_per_level)

        # Convention on levels is that 0 is for global mean, 1 is for huc2, 2 is for huc4, and 3 is for huc8.
        level_as_int = np.zeros(n_unique_combined, dtype=int)
        ptr = 0

        for i, n_codes in enumerate(unique_per_level):
            level_as_int[ptr:ptr+n_codes] = i 
            ptr += n_codes

        # Make sure exactly one row is marked as `0`
        assert np.sum(level_as_int == 0) == 1

        # We use a decreasing sequence of scale parameters
        # to encourage greater and greater shrinkage as we go down the levels
        sigma_sequence = pm.math.constant([1.0, 0.2, 0.04], dtype=FLOAT_FORMAT)
        sigma_factor = pm.HalfNormal('sigma_factor')
        beta_scales = pm.HalfNormal('beta_scales', sigma_sequence * sigma_factor, shape = [p, n_levels])
        print(f"Found {n_unique_combined} unique huc2, huc4 values. Breakdown: {unique_per_level}")

        # Create random variables with N(0,1) prior. These will be rescaled and shifted
        # to create the final coefficients later on
        u = pm.Normal('u', mu=0, sigma=beta_scales[:, level_as_int].T, dims = ["beta_hucs", "predictor"])
    
        # 1. Get integer codes for each level (ensure integer type)
        # Use .values to get numpy array, specify dtype for JAX
        codes_huc2 = final_gdf_subset['huc2'].astype('category').cat.codes.values.astype(np.int32)
        codes_huc4 = final_gdf_subset['huc4'].astype('category').cat.codes.values.astype(np.int32)
        #codes_huc8 = final_gdf_subset['huc8'].astype('category').cat.codes.values.astype(np.int32)

        # 2. Calculate the indices into the full 'u' tensor for each observation
        # Global mean index is 0 for all observations
        # HUC2 indices start after global mean (at cumulative_unique[0] = 1)
        # HUC4 indices start after HUC2 (at cumulative_unique[1])
        # HUC8 indices start after HUC4 (at cumulative_unique[2])
        indices_huc2 = codes_huc2 + cumulative_unique[0]
        indices_huc4 = codes_huc4 + cumulative_unique[1]
        #indices_huc8 = codes_huc8 + cumulative_unique[2]

        # 3. Construct beta by indexing 'u' directly and summing components
        # Start with global effect (index 0), broadcasted to all n observations
        beta_global = pt.zeros([n,p]) # Shape (1, p), will broadcast
        beta_global = pt.subtensor.inc_subtensor(beta_global[:], u[0:1, :])
        
        # Add HUC level effects by indexing the full 'u' tensor
        # u[indices_hucX] performs the advanced indexing directly
        beta = beta_global + u[indices_huc2] + u[indices_huc4] #+ u[indices_huc8]
        # Each u[indices_...] should have shape (n, p)
        # Broadcasting of beta_global handles the addition correctly

        beta = pm.Deterministic('beta', beta, dims=['obs_id', 'predictor'])
        mu = pm.math.sum(beta * X_data, axis=1)


    else:    
        intercept = pm.Normal('intercept', mu=y.mean(), sigma=y.std() * 2)
        beta_sd = pm.HalfNormal('beta_sd', sigma=5)
        beta = pm.Normal('beta', mu=0, sigma=beta_sd, dims='predictor')
        mu = intercept + X[is_used] @ beta

    # Spatial random effect using geographic coordinates
    if USE_GP:
        ell = pm.Beta('ell', alpha=2, beta=2)
        eta = pm.HalfNormal('eta', sigma=0.05) # Push it to be closer to zero
        cov_func = eta**2 * pm.gp.cov.Matern52(input_dim=2, ls=ell)
        gp = pm.gp.HSGP(m=[m, m], c=c, parametrization= "non-centered", cov_func=cov_func)
        eps = y[is_used] - mu
        f = gp.prior("f", X=coords[is_used], hsgp_coeffs_dims="basis_coeffs", gp_dims="obs_id",dtype=FLOAT_FORMAT)
        mu += f.astype(FLOAT_FORMAT)

    if USE_EMBED:
        embed_scale1 = pm.HalfNormal("embed_scale1", sigma=0.25)
        embed_scale2 = pm.HalfNormal("embed_scale2", sigma=0.25)
        beta_embed_1 = pm.Normal('beta_embed_1', mu=0, sigma=embed_scale1, shape=(embed_len, 8))
        alpha_embed_1 = pm.Normal('alpha_embed_1', mu=0, sigma=embed_scale1, shape=(1, 8))
        beta_embed_2 = pm.Normal('beta_embed_2', mu=0, sigma=embed_scale2, shape=(8, 1))

        activation_1 = pm.math.tanh(pm.math.dot(Z[is_used], beta_embed_1) + alpha_embed_1)
        mu_embed = pm.Deterministic("mu_embed", pm.math.dot(activation_1, beta_embed_2))
    else:
        mu_embed = 0.

    mu += pt.squeeze(mu_embed)
       
    # Storing `mu` can take a lot of memory; this controls
    # whether or not it is stored in the trace
    if TRACK_MU:
        pm.Deterministic('mu', mu)

    sigma = pm.HalfCauchy('sigma', beta=1)
    likelihood = pm.Normal('likelihood', mu=mu, sigma=sigma, observed=y[is_used])


Preparing to run model inference with 64220 samples and 169 predictors
Range of response values is -9.25863265991211 to 8.884825706481934
Range of predictor values is 0.0 to 1.0
Found 209 unique huc2, huc4 values. Breakdown: [1, 18, 190]


### Logp profiling

In [7]:
RUN_PROFILING = False
if RUN_PROFILING:
    hierarchical_model.profile(hierarchical_model.logp()).summary()

RUN_DEBUGGING = False
if RUN_DEBUGGING:
    print(hierarchical_model.debug(verbose=True))

### Run sampler

In [9]:
TEST_MODE = True

if TEST_MODE:
    # Set the number of draws to 5 for testing
    # and use the numpyro sampler
    tune = 2
    draws = 2
    cores = 2
    chains = 2
    trace_filename = "trace-test.nc"
    single_trace_path = "trace-test-chain{chain}.nc"
else:
    tune = 2000
    draws = 500
    cores = 1
    chains = 2
    trace_filename = "trace-revised.nc"
    single_trace_path = "trace-revised-chain{chain}.nc"
    
trace_path = DATA_DIR / trace_filename


tps = []
for chain in range(chains):
    with hierarchical_model:
        single_trace = pm.sample(
            return_inferencedata=True, 
            chains=1, 
            tune=tune, 
            draws=draws, 
            cores=cores, 
            nuts_sampler="numpyro", 
            progressbar=True,
            target_accept=0.85)

    tp = DATA_DIR / single_trace_path.format(chain=chain)
    tps += [tp]
    
    # Delete old trace if it exists
    # and make sure hdf5 file is closed
    if os.path.exists(tp):
        os.remove(tp)
    
    az.to_netcdf(single_trace, tp)

# After sampling complete across all chains, load
# chains and concat across chain dimension. This is
# to prevent OOM GPU memory issues on a machine with much more RAM.
multi_trace = az.concat([az.from_netcdf(tp) for tp in tps], dim="chain")
az.to_netcdf(multi_trace, trace_path)

Only 2 samples per chain. Reliable r-hat and ESS diagnostics require longer chains for accurate estimate.
/mnt/m2ssd/data/Dropbox/research/motives/motives-wq-modeling/.venv/lib/python3.10/site-packages/jax/_src/numpy/array_methods.py:122: UserWarning: Explicitly requested dtype float64 requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  return lax_numpy.astype(self, dtype, copy=copy, device=device)
sample: 100%|██████████| 4/4 [00:05<00:00,  1.45s/it, 1 steps of size 4.93e-01. acc. prob=0.00]
/mnt/m2ssd/data/Dropbox/research/motives/motives-wq-modeling/.venv/lib/python3.10/site-packages/jax/_src/numpy/array_methods.py:122: UserWarning: Explicitly requested dtype float64 requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_ena

PosixPath('data/trace-test.nc')

### Upload to S3

In [ ]:
print(f"Uploading trace file to S3...")
s3 = boto3.client('s3')
s3.upload_file(trace_path, bucket_name, trace_filename)
print(f"✓ Successfully uploaded trace file to S3")


Uploading trace file to S3...
✓ Successfully uploaded trace file to S3
